In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [2]:
antibiotics_url = data.burtin.url
print(antibiotics_url)

https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/burtin.json


In [3]:
antibiotics = pd.read_json(antibiotics_url)
antibiotics.head()

,Bacteria,Penicillin,Streptomycin,Neomycin,Gram_Staining,Genus
0,Aerobacter aerogenes,870.000,1.00,1.600,negative,other
1,Bacillus anthracis,0.001,0.01,0.007,positive,other
2,Brucella abortus,1.000,2.00,0.020,negative,other
3,Diplococcus pneumoniae,0.005,11.00,10.000,positive,other
4,Escherichia coli,100.000,0.40,0.100,negative,other


### Adjusting Scales in Altair

In [7]:
alt.Chart(antibiotics).mark_point(filled=True).encode(
    alt.X('Neomycin:Q', scale=alt.Scale(type='sqrt'))
)

alt.Chart(...)

In [8]:
alt.Chart(antibiotics).mark_point(filled=True).encode(
    alt.X('Neomycin:Q', scale=alt.Scale(type='linear'))
)

alt.Chart(...)

In [9]:
alt.Chart(antibiotics).mark_point(filled=True).encode(
    alt.X('Neomycin:Q', scale=alt.Scale(type='log'))
)

alt.Chart(...)

In [13]:
alt.Chart(antibiotics).mark_point(filled=True).encode(
    alt.X('Neomycin:Q', 
        sort='descending',
        scale=alt.Scale(type='log'),
        axis=alt.Axis(orient='top'),
        title='Neomycin MIC (reverse log scale)'
        )
)

alt.Chart(...)

In [25]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q', 
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='top'),
        axis=alt.Axis(tickCount=5),
        title='Neomycin MIC (reverse log scale)'
        ),
    alt.Y('Penicillin:Q',
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='right'),
        axis=alt.Axis(tickCount=5),
        title='Penicillin MIC (reverse log scale)'
    )
).properties(
    width=300,
    height=300
)

alt.Chart(...)

### Color Legends in Altair

In [34]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q', 
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='top'),
        axis=alt.Axis(tickCount=5),
        title='Neomycin MIC (reverse log scale)'
        ),
    alt.Y('Penicillin:Q',
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='right'),
        axis=alt.Axis(tickCount=5),
        title='Penicillin MIC (reverse log scale)'
    ),
    alt.Color('Gram_Staining:N',
        scale=alt.Scale(
            domain=['negative', 'positive'],
            range=['hotpink', 'purple']
        ),
        legend=None
    )
).properties(
    width=300,
    height=300
)

alt.Chart(...)

In [41]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q', 
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='top'),
        axis=alt.Axis(tickCount=5),
        title='Neomycin MIC (reverse log scale)'
        ),
    alt.Y('Penicillin:Q',
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='right'),
        axis=alt.Axis(tickCount=5),
        title='Penicillin MIC (reverse log scale)'
    ),
    alt.Color('Bacteria:O',
        scale=alt.Scale(scheme='turbo')
    )
).properties(
    width=300,
    height=300
)

alt.Chart(...)

In [48]:
x = antibiotics['Bacteria'].apply(lambda x: x.split(' ')[0])
x

0         Aerobacter
1           Bacillus
2           Brucella
3        Diplococcus
4        Escherichia
5         Klebsiella
6      Mycobacterium
7            Proteus
8        Pseudomonas
9         Salmonella
10        Salmonella
11    Staphylococcus
12    Staphylococcus
13     Streptococcus
14     Streptococcus
15     Streptococcus
Name: Bacteria, dtype: object

In [51]:
pd.Index(x).get_loc('Streptococcus')

slice(13, 16, None)

In [55]:
alt.Chart(antibiotics).mark_circle(size=80).transform_calculate(
    Split='split(datum.Bacteria, " ")[0]'
).transform_calculate(
    Genus='indexof(["Salmonella", "Staphylococcus", "Streptococcus"], datum.Split) >= 0 ? datum.Split : "Other"'
).encode(
    alt.X('Neomycin:Q', 
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='top'),
        axis=alt.Axis(tickCount=5),
        title='Neomycin MIC (reverse log scale)'
        ),
    alt.Y('Penicillin:Q',
        sort='descending',
        scale=alt.Scale(type='log', domain=[0.001, 1000]),
        # axis=alt.Axis(orient='right'),
        axis=alt.Axis(tickCount=5),
        title='Penicillin MIC (reverse log scale)'
    ),
    alt.Color('Genus:N',
        scale=alt.Scale(
            domain=["Salmonella", "Staphylococcus", "Streptococcus", "Other"],
            range=["salmon", "firebrick", "dodgerblue", "rgb(84,162,75)"]
        )
    ),
    tooltip='Bacteria'
).properties(
    width=300,
    height=300
)

alt.Chart(...)

In [123]:
alt.Chart(antibiotics).mark_rect().encode(
    alt.Y('Bacteria:N', 
        sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
        axis=alt.Axis(
            orient='right',
            titleY=0,
            titleX=7,
            titleAlign='left',
            titleAngle=0
        )
    ),
    alt.Color('Penicillin:Q',
        scale=alt.Scale(type='log', scheme='turbo', nice=True),
        legend=alt.Legend(orient='right', titleOrient='right', tickCount=5, titleLimit=300, padding=60, offset=-50),
        title='Penicillin MIC (minimum inhibitory concentration)'
    )
).properties(
    width=30,
    height=500
)

alt.Chart(...)

In [135]:
alt.Chart(antibiotics, title='Peinillin Resistance of Bacterial Strains').mark_rect().encode(
    alt.Y('Bacteria:N', 
        sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
        axis=alt.Axis(
            orient='right',
            title=None
        )
    ),
    alt.Color('Penicillin:Q',
        scale=alt.Scale(type='log', scheme='turbo', nice=True),
        legend=alt.Legend(orient='right', titleOrient='right', tickCount=5, titleLimit=300, padding=60, offset=-50),
        title='Penicillin MIC'
    )
).configure_title(anchor='start', offset=5, fontSize=16).properties(
    width=30,
    height=500
)

alt.Chart(...)